In [1]:
import urllib.request
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 

### Ссылка на главную страницу с футбольными клубами по странам

In [2]:
r = urllib.request.urlopen('https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%BB%D1%83%D0%B1%D1%8B_%D0%BF%D0%BE_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B0%D0%BC')

In [3]:
result = r.read()

In [4]:
content = result.decode('utf-8')

In [5]:
soup = BeautifulSoup(content, 'lxml') 

In [6]:
links = []

### Парсим ее и получаем ссылки на отдельные страны

In [7]:
for div in soup.find_all('div', attrs={'class':'CategoryTreeItem'}):
    link = div.find('a').get('href')
    links.append(link)

In [8]:
len(links)

180

### Проход по каждой стране

In [9]:
teams_dict = {}
empty_team_lists = [] #список для команд стран с другой структурой страниц
empty_team_lists_links = []

for link in tqdm(links):
    page = urllib.request.urlopen('https://ru.wikipedia.org/' + link) #проходим по каждой ссылке
    result = page.read()
    content = result.decode('utf-8')
    soup = BeautifulSoup(content, 'lxml')
    for h1 in soup.find_all('h1', attrs={'class':'firstHeading'}): #находим нужный элемент в html 
        name = h1.get_text() #забираем название, напр: Футбольные клубы Англии
        if name.startswith('Категория:'):
            norm_name = name.replace('Категория:', '')
            
    teams_list = []    
    for div in soup.find_all('div', attrs={'class':'mw-category-group'}): #находим все команды в этой стране
        team_links = div.find_all('a')
        for team_link in team_links:
            team = team_link.get('title')
            #print(team)
            if team.startswith('Категория:ФК'):
                norm_team = team.replace('Категория:ФК', '')
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное сообщество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                #norm_team = norm_team.strip(punctuation)
            
            elif team.startswith('Категория:') or team.startswith('Список'):
                    continue
                
            elif team.endswith('(футбольный клуб)'):
                norm_team = team.replace('(футбольный клуб)', '')
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное сообщество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                #norm_team = norm_team.strip(punctuation)
            else:
                norm_team = team
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное сообщество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                #norm_team = norm_team.strip(punctuation)
            
            teams_list.append(norm_team)
    
    teams_list1 = teams_list
    
    teams_list2 = []
    second_page_div = soup.find('div', attrs={'id':'mw-pages'})
    if second_page_div:
        a = second_page_div.find('a')
        if a.get_text() == 'Следующая страница':
            second_page = a.get('href')
            link = 'https://ru.wikipedia.org'+second_page
            page2 = urllib.request.urlopen(link)
            result2 = page2.read()
            content2 = result2.decode('utf-8')
            soup2 = BeautifulSoup(content2, 'lxml')
            h2 = soup2.find('div', attrs={'id':'mw-pages'})
            for a in h2.find_all('a'):
                team2 = a.get('title')
                #print(team2)
                if team2.startswith('Категория:ФК'):
                    norm_team2 = team2.replace('Категория:ФК', '')
                    norm_team2 = re.sub(r'[«»]','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    norm_team2 = re.sub(r'\(футбольный клуб.+\)','', norm_team2)
                    norm_team2 = re.sub(r'\(футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(спортивное общество\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб.+\)','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    #norm_team2 = norm_team2.strip(punctuation)
                elif team2.startswith('Категория:') or team2.startswith('Список'):
                    continue    
                elif team.endswith('(футбольный клуб)'):
                    norm_team2 = team2.replace('(футбольный клуб)', '')
                    norm_team2 = re.sub(r'[«»]','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    norm_team2 = re.sub(r'\(футбольный клуб.+\)','', norm_team2)
                    norm_team2 = re.sub(r'\(футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(спортивное общество\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб.+\)','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    #norm_team2 = norm_team2.strip(punctuation)
                else:
                    norm_team2 = team2
                    norm_team2 = re.sub(r'[«»]','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    norm_team2 = re.sub(r'\(футбольный клуб.+\)','', norm_team2)
                    norm_team2 = re.sub(r'\(футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(спортивное общество\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб.+\)','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    #norm_team2 = norm_team2.strip(punctuation)
                #print(norm_team2)
                
                teams_list2.append(norm_team2)
            #print(teams_list2)
    
    if teams_list2:
        full_teams_list = set(teams_list1 + teams_list2)
    else:
        full_teams_list = set(teams_list1)
                

    if full_teams_list: #если список с командами не пустой - добавляем в словарь
        teams_dict[norm_name] = full_teams_list
    else:
        empty_team_lists.append(norm_name) #если список команд пустой - добавляем в список с такими же странами
        empty_team_lists_links.append(link) 

100%|████████████████████████████████████████████████████████████████████████████████| 180/180 [02:16<00:00,  1.32it/s]


### Добавление в словарь оставшихся клубов 

In [13]:
bad_links = []

for link in tqdm(empty_team_lists_links):
    #print('https://ru.wikipedia.org' + link)
    page = urllib.request.urlopen('https://ru.wikipedia.org' + link) 
    result = page.read()
    content = result.decode('utf-8')
    soup = BeautifulSoup(content, 'lxml')
    for h1 in soup.find_all('h1', attrs={'class':'firstHeading'}): #находим нужный элемент в html 
        name = h1.get_text() #забираем название, напр: Футбольные клубы Англии
        if name.startswith('Категория:'):
            norm_name = name.replace('Категория:', '')
    
    teams_list = []    
    for div in soup.find_all('div', attrs={'class':'mw-category-generated'}): #находим все команды в этой стране
        #print(div)
        team_links = div.find_all('a')
        #print(team_links)
        for team_link in team_links:
            team = team_link.get('title')
            if team.startswith('Категория:ФК'):
                norm_team = team.replace('Категория:ФК', '') 
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                
            elif team.endswith('(футбольный клуб)'):
                norm_team = team.replace('(футбольный клуб)', '')
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                
            elif team.startswith('Категория:'):
                bad_links.append(team_link.get('href'))
                #norm_team = norm_team.strip(' ')
                
            else:
                norm_team = team
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                    
            teams_list.append(norm_team)
    teams_list = set(teams_list)
    teams_dict[norm_name] = teams_list
            
    

100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [00:29<00:00,  2.64it/s]


### Парсинг клубов отдельной страны/города по ссылке 

In [15]:
link = 'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%BB%D1%83%D0%B1%D1%8B_%D0%9A%D0%B8%D0%B5%D0%B2%D0%B0'
page = urllib.request.urlopen(link) 
result = page.read()
content = result.decode('utf-8')
soup = BeautifulSoup(content, 'lxml')
for h1 in soup.find_all('h1', attrs={'class':'firstHeading'}): #находим нужный элемент в html 
        name = h1.get_text() #забираем название, напр: Футбольные клубы Англии
        if name.startswith('Категория:'):
            norm_name = name.replace('Категория:', '')
            
teams_list = []    
for div in soup.find_all('div', attrs={'class':'mw-category-generated'}): #находим все команды в этой стране
    #print(div)
    team_links = div.find_all('a')
    #print(team_links)
    for team_link in team_links:
        team = team_link.get('title')
        if team.startswith('Категория:ФК'):
            norm_team = team.replace('Категория:ФК', '') 
            norm_team = re.sub(r'[«»]','', norm_team)
            norm_team = norm_team.strip(' ')
            norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
            norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
            norm_team = norm_team.strip(' ')
                
        elif team.endswith('(футбольный клуб)'):
            norm_team = team.replace('(футбольный клуб)', '')
            norm_team = re.sub(r'[«»]','', norm_team)
            norm_team = norm_team.strip(' ')
            norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
            norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
            norm_team = norm_team.strip(' ')
                
        elif team.startswith('Категория:'):
            bad_links.append(team_link.get('href'))
            #norm_team = norm_team.strip(' ')
                
        else:
            norm_team = team
            norm_team = re.sub(r'[«»]','', norm_team)
            norm_team = norm_team.strip(' ')
            norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
            norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
            norm_team = norm_team.strip(' ')
                    
        teams_list.append(norm_team)
teams_list = set(teams_list)
teams_dict[norm_name] = teams_list


In [18]:
list_teams = [*teams_dict.values()]

In [23]:
count_teams = []
for team_set in list_teams:
    for team in team_set:
        count_teams.append(team)

In [33]:
uniq_teams = set(count_teams)

### Мэтч по своему списку команд

In [26]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from nltk import word_tokenize
from collections import Counter
from string import punctuation
punct = punctuation+'©«»—…“”*№–'

In [ ]:
df = pd.read_csv('result_tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

In [28]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [30]:
def which_language (text):
    if bool(re.search('[\u0400-\u04FF]', text)) == True:
        return 'russian'
    else:
        return 'english'
    
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
    lang.append(which_language(str(text)))
    
df['Language'] = lang
grouped=df.groupby('Language')
for name, group in grouped: 
    if name == 'english': 
        english = group.drop('Language',axis=1) 
    else: 
        russian = group.drop('Language',axis=1)


In [31]:
russian = russian.reset_index(drop=True)

In [35]:
teams1_match = np.zeros(russian.shape[0])
teams2_match = np.zeros(russian.shape[0])

In [36]:
for i, text in enumerate(russian['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for txt in txt_tokens:
        count_tokens = []
        for token in txt:
            token = token.strip(punct)
            if token in uniq_teams:
                count_tokens.append(token)
        c = Counter(count_tokens)
        
        try: 
            max_w1 = sorted(c, key=c.get)[-1]
        except:
            continue
        
        try:
            max_w2 = sorted(c, key=c.get)[-2]

        except:
            continue
        
        if russian['team_of_season_1_name_in_russian'][i] == max_w1 or russian['team_of_season_1_name_in_russian'][i] == max_w2:
            teams1_match[i] = 1

        
        if russian['team_of_season_2_name_in_russian'][i] == max_w1 or russian['team_of_season_2_name_in_russian'][i] == max_w2:
            teams2_match[i] = 1

In [37]:
russian['list_direct_match_for_teams1'] = teams1_match
russian['list_direct_match_for_teams2'] = teams2_match

In [38]:
success1 = russian[russian['list_direct_match_for_teams1'] == 1].shape[0]
success2 = russian[russian['list_direct_match_for_teams2'] == 1].shape[0]

all_vals = russian['list_direct_match_for_teams1'].shape[0]

result_team1 = success1 / all_vals
print(result_team1)

result_team2 = success2 / all_vals
print(result_team2)

0.17921537896165832
0.1814253663595282


In [39]:
russian.to_csv(r'C:\Users\User\Desktop\project\russian_custom_lists_and_directmatch.csv', index = False)